In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
api_key_openai = os.environ.get('OPENAI_API_KEY')
org_openai = os.environ.get('OPENAI_ORG')

import openai
openai.organization = org_openai

In [2]:
import os
import sys
import inspect

sys.path.insert(0, '../..')

import pickle

In [29]:
from src.GetEmbeddings.GetEmbeddingsOpenAI import GetEmbeddingsOpenAI

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[autoreload of ExtractTopWords failed: Traceback (most recent call last):
  File "c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\arik_\anaconda3\envs\llm_sem\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<

In [3]:
from src.Clustering.Clustering import Clustering_and_DimRed

c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\arik_\anaconda3\envs\llm_sem\Lib\site-packages\umap\distances.py:1086: NumbaD

In [4]:
data = pickle.load(open("../../Data/Emebeddings/embeddings_reuters_raw.pkl", "rb"))
embeddings = data["embeddings"]
corpus = data["corpus"]

In [5]:
clusterer = Clustering_and_DimRed()
dim_red_embeddings, clusters = clusterer.cluster_and_reduce(embeddings)

UMAP(angular_rp_forest=True, metric='cosine', min_dist=0, n_components=5, random_state=42, verbose=True)
Fri Jul 28 17:12:17 2023 Construct fuzzy simplicial set
Fri Jul 28 17:12:17 2023 Finding Nearest Neighbors
Fri Jul 28 17:12:17 2023 Building RP forest with 10 trees
Fri Jul 28 17:12:25 2023 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	 6  /  13
	Stopping threshold met -- exiting after 6 iterations
Fri Jul 28 17:12:52 2023 Finished Nearest Neighbor Search
Fri Jul 28 17:12:55 2023 Construct embedding


Epochs completed: 100%| ██████████ 200/200 [00:09]


Fri Jul 28 17:13:05 2023 Finished embedding


In [6]:
import ExtractTopWords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
extractor = ExtractTopWords.ExtractTopWords()

In [8]:
extractor.extract_centroids(dim_red_embeddings, clusters)

{0: array([-3.7164304,  3.9222953,  5.908199 ,  6.075133 ,  4.827653 ],
       dtype=float32),
 1: array([-3.7654023,  4.2164207,  5.8612056,  6.4997573,  4.904274 ],
       dtype=float32),
 2: array([ 5.910965 ,  2.4154532,  4.8060503,  5.771162 , 11.888593 ],
       dtype=float32),
 3: array([10.06639  ,  4.091257 ,  5.499461 ,  7.893647 ,  2.2268891],
       dtype=float32),
 4: array([12.484892 ,  7.8714714,  9.038342 ,  6.1947913,  2.958478 ],
       dtype=float32),
 5: array([4.695055 , 6.4599414, 8.006299 , 6.198267 , 2.410783 ],
       dtype=float32),
 6: array([9.630017 , 6.8125625, 8.012566 , 4.08587  , 5.3370237],
       dtype=float32),
 7: array([4.4590163, 5.2563252, 8.423323 , 6.6133666, 5.0430827],
       dtype=float32),
 8: array([6.745958 , 3.4827986, 6.3263617, 6.8663306, 4.501375 ],
       dtype=float32),
 9: array([12.940467 , -3.6038241,  8.827755 ,  6.4337673,  6.571346 ],
       dtype=float32),
 10: array([12.917635 , -3.469067 ,  8.776229 ,  6.5808606,  6.401622 

In [9]:
extractor.compute_centroid_similarity(dim_red_embeddings, extractor.extract_centroids(dim_red_embeddings, clusters), 0)

array([0.00458414, 0.00517667, 0.00517025, ..., 0.00498587, 0.00548446,
       0.00513136], dtype=float32)

In [36]:
vocab = extractor.compute_corpus_vocab(corpus)

c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Processing corpus: 100%|██████████| 10788/10788 [00:14<00:00, 745.13it/s] 


In [34]:
words_per_topic = extractor.compute_words_topics(corpus, vocab, clusters)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Computing words per topic: 100%|██████████| 10788/10788 [00:07<00:00, 1472.01it/s]


In [37]:
len(vocab)

9181

In [41]:
embeddings_vocab = extractor.embed_vocab_openAI(api_key= api_key_openai, vocab = vocab)

c:\Users\arik_\Documents\Dokumente\Studium\Master\LLM_seminar\Project\src


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 9181/9181 [46:23<00:00,  3.30it/s]  


TypeError: string indices must be integers, not 'str'